## Importing Library

In [95]:
import numpy as np 
import pandas as pd 
import nltk
import os
import gc
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
#pd.set_option('display.max_colwidth',100)
pd.set_option('display.max_colwidth', -1)

In [96]:
gc.collect()

1220

### Loading dataset and basic visualization

In [97]:
train=pd.read_csv('./Desktop/Russell_PycharmML/KaggleTomato/train.tsv',sep='\t')
print(train.shape)
train.head()

(156060, 4)


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [98]:
test=pd.read_csv('./Desktop/Russell_PycharmML/KaggleTomato/test.tsv',sep='\t')
print(test.shape)
test.head()

(66292, 3)


,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine effort .
1,156062,8545,An intermittently pleasing but mostly routine effort
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [99]:
sub=pd.read_csv('./Desktop/Russell_PycharmML/KaggleTomato/sampleSubmission.csv')
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


**Adding Sentiment column to test datset and joing train and test for preprocessing**

In [100]:
test['Sentiment']=-999
test.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,156061,8545,An intermittently pleasing but mostly routine effort .,-999
1,156062,8545,An intermittently pleasing but mostly routine effort,-999
2,156063,8545,An,-999
3,156064,8545,intermittently pleasing but mostly routine effort,-999
4,156065,8545,intermittently pleasing but mostly routine,-999


In [101]:
df=pd.concat([train,test],ignore_index=True)
print(df.shape)
df.tail()

(222352, 4)


,PhraseId,SentenceId,Phrase,Sentiment
222347,222348,11855,"A long-winded , predictable scenario .",-999
222348,222349,11855,"A long-winded , predictable scenario",-999
222349,222350,11855,"A long-winded ,",-999
222350,222351,11855,A long-winded,-999
222351,222352,11855,predictable scenario,-999


In [102]:
del train,test
gc.collect()

204

** cleaning review**

In [103]:
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation
import re


In [104]:
def clean_review(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        review=re.sub('[^a-zA-Z]',' ',review)
        #review=[stemmer.stem(w) for w in word_tokenize(str(review).lower())]
        review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review=' '.join(review)
        review_corpus.append(review)
    return review_corpus

In [105]:
df['clean_review']=clean_review(df.Phrase.values)
df.head()


,PhraseId,SentenceId,Phrase,Sentiment,clean_review
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1,a series of escapade demonstrating the adage that what is good for the goose is also good for the gander some of which occasionally amuses but none of which amount to much of a story
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2,a series of escapade demonstrating the adage that what is good for the goose
2,3,1,A series,2,a series
3,4,1,A,2,a
4,5,1,series,2,series


** seperating train and test dataset**

In [106]:
df_train=df[df.Sentiment!=-999]
df_train.shape

(156060, 5)

In [107]:
df_test=df[df.Sentiment==-999]
df_test.drop('Sentiment',axis=1,inplace=True)
print(df_test.shape)
df_test.head()

(66292, 4)


,PhraseId,SentenceId,Phrase,clean_review
156060,156061,8545,An intermittently pleasing but mostly routine effort .,an intermittently pleasing but mostly routine effort
156061,156062,8545,An intermittently pleasing but mostly routine effort,an intermittently pleasing but mostly routine effort
156062,156063,8545,An,an
156063,156064,8545,intermittently pleasing but mostly routine effort,intermittently pleasing but mostly routine effort
156064,156065,8545,intermittently pleasing but mostly routine,intermittently pleasing but mostly routine


In [108]:
del df
gc.collect()

112

### Splitting Train dataset into train and 20% validation set

In [109]:
train_text=df_train.clean_review.values
test_text=df_test.clean_review.values
target=df_train.Sentiment.values
y=to_categorical(target)
print(train_text.shape,target.shape,y.shape)

(156060,) (156060,) (156060, 5)


In [110]:
X_train_text,X_val_text,y_train,y_val=train_test_split(train_text,y,test_size=0.2,stratify=y,random_state=123)
print(X_train_text.shape,y_train.shape)
print(X_val_text.shape,y_val.shape)

(124848,) (124848, 5)
(31212,) (31212, 5)


### Finding number of unique words in train set

In [111]:
all_words=' '.join(X_train_text)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
num_unique_word

13732

### Finding max length of a review in train set

In [112]:
r_len=[]
for text in X_train_text:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)
    
MAX_REVIEW_LEN=np.max(r_len)
MAX_REVIEW_LEN

48

## Building Keras LSTM model

In [113]:
max_features = num_unique_word
max_words = MAX_REVIEW_LEN
batch_size = 128
epochs = 3
num_classes=5

** Tokenize Text**

In [114]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train_text))
X_train = tokenizer.texts_to_sequences(X_train_text)
X_val = tokenizer.texts_to_sequences(X_val_text)
X_test = tokenizer.texts_to_sequences(test_text)

** sequence padding**

In [115]:
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
print(X_train.shape,X_val.shape,X_test.shape)

(124848, 48) (31212, 48) (66292, 48)


## 1. LSTM model

In [116]:
model1=Sequential()
model1.add(Embedding(max_features,100,mask_zero=True))
model1.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model1.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model1.add(Dense(num_classes,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         1373200   
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 64)          42240     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 165       
Total params: 1,428,021
Trainable params: 1,428,021
Non-trainable params: 0
_________________________________________________________________


In [117]:
%%time
history1=model1.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3


   128/124848 [..............................] - ETA: 18:06 - loss: 1.6091 - acc: 0.1641

   256/124848 [..............................] - ETA: 9:48 - loss: 1.6085 - acc: 0.2383 

   384/124848 [..............................] - ETA: 7:00 - loss: 1.6074 - acc: 0.2812

   512/124848 [..............................] - ETA: 5:37 - loss: 1.6063 - acc: 0.2930

...

124544/124848 [============================>.] - ETA: 0s - loss: 0.7785 - acc: 0.6815

124672/124848 [============================>.] - ETA: 0s - loss: 0.7785 - acc: 0.6814

124800/124848 [============================>.] - ETA: 0s - loss: 0.7786 - acc: 0.6814

124848/124848 [==============================] - 97s 778us/step - loss: 0.7785 - acc: 0.6814 - val_loss: 0.8242 - val_acc: 0.6610


CPU times: user 21min 55s, sys: 4min 16s, total: 26min 11s
Wall time: 4min 54s


In [118]:
y_pred1=model1.predict_classes(X_test,verbose=1)

   32/66292 [..............................] - ETA: 7:02

  192/66292 [..............................] - ETA: 1:31

  320/66292 [..............................] - ETA: 1:06

  448/66292 [..............................] - ETA: 55s 

  576/66292 [..............................] - ETA: 48s

...

65824/66292 [============================>.] - ETA: 0s

65952/66292 [============================>.] - ETA: 0s

66080/66292 [============================>.] - ETA: 0s

66208/66292 [============================>.] - ETA: 0s

66292/66292 [==============================] - 27s 406us/step


In [119]:
sub.Sentiment=y_pred1
sub.to_csv('sub1.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


## 2. CNN

In [120]:
model2= Sequential()
model2.add(Embedding(max_features,100,input_length=max_words))
model2.add(Dropout(0.2))
model2.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model2.add(GlobalMaxPooling1D())
model2.add(Dense(128,activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(num_classes,activation='softmax'))
model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 48, 100)           1373200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 100)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 48, 64)            19264     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 645       
Total para

In [121]:
%%time
history2=model2.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3


   128/124848 [..............................] - ETA: 6:40 - loss: 1.6234 - acc: 0.0938

   384/124848 [..............................] - ETA: 2:38 - loss: 1.6092 - acc: 0.1667

   640/124848 [..............................] - ETA: 1:47 - loss: 1.5914 - acc: 0.3094

   896/124848 [..............................] - ETA: 1:26 - loss: 1.5747 - acc: 0.3571

...

124288/124848 [============================>.] - ETA: 0s - loss: 0.7044 - acc: 0.7081

124544/124848 [============================>.] - ETA: 0s - loss: 0.7045 - acc: 0.7080

124800/124848 [============================>.] - ETA: 0s - loss: 0.7045 - acc: 0.7080

124848/124848 [==============================] - 35s 281us/step - loss: 0.7044 - acc: 0.7081 - val_loss: 0.8090 - val_acc: 0.6667


CPU times: user 7min 59s, sys: 1min 55s, total: 9min 55s
Wall time: 1min 45s


In [122]:
y_pred2=model2.predict_classes(X_test, verbose=1)

   32/66292 [..............................] - ETA: 3:28

 1344/66292 [..............................] - ETA: 7s  

 2656/66292 [>.............................] - ETA: 4s

 4000/66292 [>.............................] - ETA: 3s

 5344/66292 [=>............................] - ETA: 3s

...

62304/66292 [===========================>..] - ETA: 0s

63584/66292 [===========================>..] - ETA: 0s

64928/66292 [============================>.] - ETA: 0s

66272/66292 [============================>.] - ETA: 0s

66292/66292 [==============================] - 3s 40us/step


In [123]:
sub.Sentiment=y_pred2
sub.to_csv('sub2.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,3
1,156062,3
2,156063,2
3,156064,2
4,156065,3


## 3. CNN +GRU

In [124]:
model3= Sequential()
model3.add(Embedding(max_features,100,input_length=max_words))
model3.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.25))
model3.add(GRU(128,return_sequences=True))
model3.add(Dropout(0.3))
model3.add(Flatten())
model3.add(Dense(128,activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(5,activation='softmax'))
model3.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 48, 100)           1373200   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 48, 64)            19264     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 24, 64)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 24, 64)            0         
_________________________________________________________________
gru_1 (GRU)                  (None, 24, 128)           74112     
_________________________________________________________________
dropout_4 (Dropout)          (None, 24, 128)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3072)              0         
__________

In [125]:
%%time
history3=model3.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3


   128/124848 [..............................] - ETA: 13:08 - loss: 1.6087 - acc: 0.2109

   256/124848 [..............................] - ETA: 7:06 - loss: 1.5784 - acc: 0.3438 

   384/124848 [..............................] - ETA: 5:04 - loss: 1.5353 - acc: 0.3984

   512/124848 [..............................] - ETA: 4:03 - loss: 1.4898 - acc: 0.4297

...

124544/124848 [============================>.] - ETA: 0s - loss: 0.7298 - acc: 0.6952

124672/124848 [============================>.] - ETA: 0s - loss: 0.7299 - acc: 0.6952

124800/124848 [============================>.] - ETA: 0s - loss: 0.7298 - acc: 0.6952

124848/124848 [==============================] - 65s 518us/step - loss: 0.7298 - acc: 0.6952 - val_loss: 0.8120 - val_acc: 0.6715


CPU times: user 14min 55s, sys: 2min 54s, total: 17min 50s
Wall time: 3min 14s


In [126]:
y_pred3=model3.predict_classes(X_test, verbose=1)


   32/66292 [..............................] - ETA: 5:25

  320/66292 [..............................] - ETA: 42s 

  640/66292 [..............................] - ETA: 26s

  960/66292 [..............................] - ETA: 21s

 1248/66292 [..............................] - ETA: 19s

...

65216/66292 [============================>.] - ETA: 0s

65536/66292 [============================>.] - ETA: 0s

65856/66292 [============================>.] - ETA: 0s

66176/66292 [============================>.] - ETA: 0s

66292/66292 [==============================] - 11s 172us/step


In [127]:
sub.Sentiment=y_pred3
sub.to_csv('sub3.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


## 4. Bidirectional GRU

In [128]:
model4 = Sequential()

model4.add(Embedding(max_features, 100, input_length=max_words))
model4.add(SpatialDropout1D(0.25))
model4.add(Bidirectional(GRU(128)))
model4.add(Dropout(0.5))

model4.add(Dense(5, activation='softmax'))
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 48, 100)           1373200   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 48, 100)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               175872    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 1285      
Total params: 1,550,357
Trainable params: 1,550,357
Non-trainable params: 0
_________________________________________________________________


In [129]:
%%time
history4=model4.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3


   128/124848 [..............................] - ETA: 16:54 - loss: 1.6182 - acc: 0.1484

   256/124848 [..............................] - ETA: 9:21 - loss: 1.6086 - acc: 0.2422 

   384/124848 [..............................] - ETA: 6:51 - loss: 1.5937 - acc: 0.3151

   512/124848 [..............................] - ETA: 5:35 - loss: 1.5785 - acc: 0.3691

...

124544/124848 [============================>.] - ETA: 0s - loss: 0.7349 - acc: 0.6952

124672/124848 [============================>.] - ETA: 0s - loss: 0.7350 - acc: 0.6951

124800/124848 [============================>.] - ETA: 0s - loss: 0.7351 - acc: 0.6951

124848/124848 [==============================] - 117s 933us/step - loss: 0.7351 - acc: 0.6951 - val_loss: 0.8082 - val_acc: 0.6710


CPU times: user 31min 27s, sys: 4min 33s, total: 36min 1s
Wall time: 5min 50s


In [130]:
y_pred4=model4.predict_classes(X_test, verbose=1)

   32/66292 [..............................] - ETA: 10:21

  224/66292 [..............................] - ETA: 1:45 

  416/66292 [..............................] - ETA: 1:05

  608/66292 [..............................] - ETA: 50s 

  800/66292 [..............................] - ETA: 42s

...

65536/66292 [============================>.] - ETA: 0s

65728/66292 [============================>.] - ETA: 0s

65920/66292 [============================>.] - ETA: 0s

66112/66292 [============================>.] - ETA: 0s

66292/66292 [==============================] - 19s 290us/step


In [131]:
sub.Sentiment=y_pred4
sub.to_csv('sub4.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


## 5. Glove word embedding

In [143]:
# def get_coefs(word, *arr):
#     return word, np.asarray(arr, dtype='float32')
#     
# def get_embed_mat(EMBEDDING_FILE, max_features,embed_dim):
#     # word vectors
#     embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE, encoding='utf8'))
#     print('Found %s word vectors.' % len(embeddings_index))
# 
#     # embedding matrix
#     word_index = tokenizer.word_index
#     num_words = min(max_features, len(word_index) + 1)
#     all_embs = np.stack(embeddings_index.values()) #for random init
#     embedding_matrix = np.random.normal(all_embs.mean(), all_embs.std(), 
#                                         (num_words, embed_dim))
#     for word, i in word_index.items():
#         if i >= max_features:
#             continue
#         embedding_vector = embeddings_index.get(word)
#         if embedding_vector is not None:
#             embedding_matrix[i] = embedding_vector
#     max_features = embedding_matrix.shape[0]
#     
#     return embedding_matrix

In [144]:
# # embedding matrix
# EMBEDDING_FILE = '../input/glove6b100dtxt/glove.6B.100d.txt'
# embed_dim = 100 #word vector dim
# embedding_matrix = get_embed_mat(EMBEDDING_FILE,max_features,embed_dim)
# print(embedding_matrix.shape)

FileNotFoundError: [Errno 2] No such file or directory: '../input/glove6b100dtxt/glove.6B.100d.txt'

In [134]:
# model5 = Sequential()
# model5.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True))
# model5.add(SpatialDropout1D(0.25))
# model5.add(Bidirectional(GRU(128,return_sequences=True)))
# model5.add(Bidirectional(GRU(64,return_sequences=False)))
# model5.add(Dropout(0.5))
# model5.add(Dense(num_classes, activation='softmax'))
# model5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model5.summary()

NameError: name 'embedding_matrix' is not defined

In [135]:
# %%time
# history5=model5.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=4, batch_size=batch_size, verbose=1)

RuntimeError: The model needs to be compiled before being used.

In [136]:
# y_pred5=model5.predict_classes(X_test, verbose=1)

TypeError: Sequential model cannot be built: model is empty. Add some layers first.

In [137]:
# sub.Sentiment=y_pred5
# sub.to_csv('sub5.csv',index=False)
# sub.head()

NameError: name 'y_pred5' is not defined

###  combine all output

In [145]:
sub_all=pd.DataFrame({'model1':y_pred1,'model2':y_pred2,'model3':y_pred3,'model4':y_pred4})#,'model5':y_pred5
pred_mode=sub_all.agg('mode',axis=1)[0].values
sub_all.head()

,model1,model2,model3,model4
0,2,3,2,2
1,2,3,2,2
2,2,2,2,2
3,2,2,2,2
4,2,3,2,2


In [147]:
pred_mode=[int(i) for i in pred_mode]

In [148]:
sub.Sentiment=pred_mode
sub.to_csv('./Desktop/Russell_PycharmML/KaggleTomato/tomatoSubmission.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2
